# Python program to solve the n puzzle problem

Provide necessary inputs to the program. Do not
statically mention any of them inside the program.

Input and output states shall be written in the
same file and you can read them properly as required.

Intermediate output shall be properly displayed
and will be written in a separate output file.

In [ ]:
import sys

sys.setrecursionlimit(3000)

class Node:
    def __init__(self, state, parent, move):
        self.state = state
        self.parent = parent
        self.move = move

def find_blank(state):
    for r, row in enumerate(state):
        for c, val in enumerate(row):
            if val == 0:
                return (r, c)
    return None

def move(state, direction):
    blank_pos = find_blank(state)
    if blank_pos is None:
        return None
    r, c = blank_pos
    n = len(state)
    new_state = [row[:] for row in state]

    dr = {'U': -1, 'D': 1, 'L': 0, 'R': 0}
    dc = {'U': 0, 'D': 0, 'L': -1, 'R': 1}

    new_r, new_c = r + dr[direction], c + dc[direction]

    if 0 <= new_r < n and 0 <= new_c < n:
        new_state[r][c], new_state[new_r][new_c] = new_state[new_r][new_c], new_state[r][c]
        return new_state
    else:
        return None

def state_to_tuple(state):
    return tuple(map(tuple, state))

def count_inversions(flat_list):
    inversions = 0
    for i in range(len(flat_list)):
        for j in range(i + 1, len(flat_list)):
            if flat_list[i] != 0 and flat_list[j] != 0 and flat_list[i] > flat_list[j]:
                inversions += 1
    return inversions

def is_solvable(state):
    n = len(state)
    flat_list = [num for row in state for num in row]
    inversions = count_inversions(flat_list)

    if n % 2 != 0:
        return inversions % 2 == 0
    else:
        blank_pos = find_blank(state)
        if not blank_pos:
            return False
        blank_row = blank_pos[0]
        blank_row_from_bottom = n - blank_row
        if blank_row_from_bottom % 2 != 0:
            return inversions % 2 == 0
        else:
            return inversions % 2 != 0

def dfs(current_node, goal_state, visited, max_depth, intermediate_file, depth=0):
    # Write only to file
    log_message = f"Visiting at depth {depth}:\n"
    for row in current_node.state:
        log_message += " ".join(map(str, row)) + "\n"
    log_message += "-"*10 + "\n"
    intermediate_file.write(log_message)

    if current_node.state == goal_state:
        return current_node

    if depth >= max_depth:
        return None

    visited.add(state_to_tuple(current_node.state))

    moves = {'U': 'Up', 'D': 'Down', 'L': 'Left', 'R': 'Right'}
    for direction_code, move_name in moves.items():
        new_state_matrix = move(current_node.state, direction_code)

        if new_state_matrix and state_to_tuple(new_state_matrix) not in visited:
            new_node = Node(new_state_matrix, current_node, move_name)
            result = dfs(new_node, goal_state, visited, max_depth, intermediate_file, depth + 1)
            if result:
                return result
    return None

def get_user_matrix(n, prompt):
    matrix = []
    for i in range(n):
        row_input = input(f"{prompt} - Enter row {i+1} (space-separated): ")
        row = list(map(int, row_input.split()))
        matrix.append(row)
    return matrix

def validate_state(state, n):
    flat_list = [num for row in state for num in row]
    expected_nums = set(range(n * n))
    return set(flat_list) == expected_nums

def main():
    try:
        n = int(input("Enter puzzle size (e.g., 3 for 8-puzzle, 4 for 15-puzzle): "))
        if n < 2:
            print("Puzzle size must be at least 2.")
            return
    except ValueError:
        print("Invalid input. Please enter an integer.")
        return

    initial_state = get_user_matrix(n, "Initial state (0 = blank)")
    goal_state = get_user_matrix(n, "Goal state")

    if not validate_state(initial_state, n):
        print("Invalid initial state.")
        return
    if not validate_state(goal_state, n):
        print("Invalid goal state.")
        return
    if not is_solvable(initial_state):
        print("The provided initial state is NOT solvable.")
        return

    start_node = Node(initial_state, None, None)
    visited_states = set()
    max_recursion_depth = 30

    solution_node = None
    with open("n_puzzle_intermediate_steps.txt", "w") as intermediate_file:
        solution_node = dfs(start_node, goal_state, visited_states, max_recursion_depth, intermediate_file)

    final_output_lines = []
    final_output_lines.append("--- Initial State ---")
    for row in initial_state:
        final_output_lines.append(" ".join(map(str, row)))

    final_output_lines.append("\n--- Goal State ---")
    for row in goal_state:
        final_output_lines.append(" ".join(map(str, row)))

    if solution_node:
        path = []
        current = solution_node
        while current:
            path.append((current.state, current.move))
            current = current.parent
        path.reverse()

        final_output_lines.append("\n\n--- Solution Path ---")
        for i, (state, move) in enumerate(path):
            if i == 0:
                final_output_lines.append("Step 0: Initial State")
            else:
                final_output_lines.append(f"\nStep {i}: Move {move}")
            for row in state:
                final_output_lines.append(" ".join(map(str, row)))

        success = True
    else:
        final_output_lines.append(f"\n\n--- Result ---\nNo solution found within depth {max_recursion_depth}")
        success = False

    with open("n_puzzle_results.txt", "w") as f:
        f.write("\n".join(final_output_lines))

    # Only final confirmation
    print("Generated files: 'n_puzzle_intermediate_steps.txt', 'n_puzzle_results.txt'")
    if success:
        print("Solution was found and saved.")
    else:
        print("No solution found within max depth.")

if __name__ == "__main__":
    main()


Enter puzzle size (e.g., 3 for 8-puzzle, 4 for 15-puzzle): 3
Initial state (0 = blank) - Enter row 1 (space-separated): 1 2 3
Initial state (0 = blank) - Enter row 2 (space-separated): 5 6 0
Initial state (0 = blank) - Enter row 3 (space-separated): 7 8 4
Goal state - Enter row 1 (space-separated): 1 2 3
Goal state - Enter row 2 (space-separated): 5 8 6
Goal state - Enter row 3 (space-separated): 0 7 4
Generated files: 'n_puzzle_intermediate_steps.txt', 'n_puzzle_results.txt'
Solution was found and saved.
